In [174]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import pandas as pd
import yfinance as yf
import datetime 

In [3]:
pip install yfinance

     |████████████████████████████████| 6.3MB 33.2MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=6d542956aa64dd26436a851436e8dcb2c87fdb7f96f9c31de950e707bd08ceac
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [255]:
start = datetime.datetime(2020,6,21) 
Amazon = yf.Ticker("ibm.de") 
df = Amazon.history(start=start)

In [256]:
if(len(df)==0):
  print('STOCKS not found')

In [257]:
df = df.dropna(axis=0)

In [258]:
df = df.reset_index()

In [259]:
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [260]:
data = df['Open'].copy()

In [261]:
train_data = data.copy()

In [262]:
timeseries_data = train_data.to_list()

In [263]:
n_steps = 10
X, y = prepare_data(timeseries_data, n_steps)

In [264]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [265]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=25, verbose=1)

Epoch 1/25
8/8 [==============================] - 2s 44ms/step - loss: 8931.6035
Epoch 2/25
8/8 [==============================] - 0s 41ms/step - loss: 3314.3630
Epoch 3/25
8/8 [==============================] - 0s 48ms/step - loss: 309.6376
Epoch 4/25
8/8 [==============================] - 0s 42ms/step - loss: 84.7389
Epoch 5/25
8/8 [==============================] - 0s 42ms/step - loss: 27.0286
Epoch 6/25
8/8 [==============================] - 0s 44ms/step - loss: 17.9105
Epoch 7/25
8/8 [==============================] - 0s 39ms/step - loss: 12.4940
Epoch 8/25
8/8 [==============================] - 0s 41ms/step - loss: 11.3066
Epoch 9/25
8/8 [==============================] - 0s 42ms/step - loss: 10.0185
Epoch 10/25
8/8 [==============================] - 0s 43ms/step - loss: 8.9121
Epoch 11/25
8/8 [==============================] - 0s 41ms/step - loss: 7.7867
Epoch 12/25
8/8 [==============================] - 0s 42ms/step - loss: 7.4084
Epoch 13/25
8/8 [==============================

In [266]:
def predict_next(x_input,length,days):
    x_input = np.array(x_input)
    temp_input=list(x_input)
    lst_output=[]
    i=0
    while(i<days):
        
        if(len(temp_input)>length):
            x_input=np.array(temp_input[1:])
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            temp_input.append(yhat[0][0])
            temp_input=temp_input[1:]
            lst_output.append(yhat[0][0])
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            print(yhat[0])
            temp_input.append(yhat[0][0])
            lst_output.append(yhat[0][0])
            i=i+1
        
    return lst_output

In [267]:
test_data = timeseries_data[len(data)-10:len(data)]

In [268]:
outs = predict_next(test_data,10,10)

[123.076485]


In [269]:
outs

[123.076485,
 124.39179,
 125.18774,
 125.9026,
 126.9352,
 127.90095,
 128.73425,
 129.47235,
 130.21321,
 130.96574]